In [5]:
from module import *

In [6]:
env = ConnectFourEnvironment()
input_size = 6 * 7  # Adjust according to your state size
output_size = 7  # Assuming your output is the number of valid actions
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
agent = DQNAgent(input_size, output_size, device)

num_episodes = 50000
epsilon = 0.9
target_update_frequency = 100

# Training loop
for episode in tqdm(range(num_episodes)):
    state = env.get_state()
    done = False

    while not done:
        action = agent.select_action(state, env, epsilon - episode * epsilon / num_episodes)
        # print("Action:", action)
        # env.display_board()
        next_state, reward, done = env.step(action)
        # print("Done:", done)
        agent.store_transition(Transition(state, action, next_state, reward))
        agent.train()

        state = next_state

    if episode % target_update_frequency == 0:
        agent.update_target_network()
        
    env.reset()

torch.save(agent.policy_net.state_dict(), f"{num_episodes}.pth")

 70%|███████   | 35198/50000 [52:29<27:04,  9.11it/s]  